## README

Please check your Colab GPU. You should have at least 16GB RAM to fine tune the model!

In [ ]:
!nvidia-smi

## INSTALL TRANSFORMERS / CUDA / APEX

In [ ]:
!pip3 install transformers==2.8.0

In [ ]:
!wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/pretrain_transformers.py
!wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py

In [ ]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

In [ ]:
!sh setup.sh

## GET DATA



In [ ]:
!rm -rf ruGPT3
!rm -rf sample_data
!git clone https://github.com/gotzmann/ruGPT3.git

In [ ]:
import glob
datasets = glob.glob("ruGPT3/datasets/*")
with open('corpus.txt', 'w') as outfile:
    for name in datasets:
        with open(name) as infile:
            outfile.write(infile.read())
            outfile.write("\n\n")

## PREPROCESSING

In [ ]:
data = open('corpus.txt', 'r').read()
data = data.split('\n\n')

# Do we need some preprocessing here?

# data = data.replace('COVID-19', 'коронавирус')
# data = data.replace('COVID', 'коронавирус')

# Do split the whole dataset to train / validate portions?

# import numpy as np
# import random

# random.seed(1980)
# np.random.seed(1980)

# val_ind = random.sample(range(len(data)), int(len(data) / 5))
# train = [data[i] for i in range(len(data)) if i not in val_ind][:len(data)]
# valid = [data[i] for i in range(len(data)) if i in val_ind]
# train = [str.replace('\n', ' ') for str in train]
# valid = [str.replace('\n', ' ') for str in valid]

# Or just train on the whole data?

train = [str.replace('\n', ' ') for str in data]
valid = [str.replace('\n', ' ') for str in data]

# Do we need JSON for GPT3 Large and plaintext for Medium / Small?

# train = [str.replace('"', '\\"') for str in train]
# valid = [str.replace('"', '\"') for str in valid]

# train = ['{ "text" : "' + str + '" }' for str in train]
# valid = ['{ "text" : "' + str + '" }' for str in valid]

with open("train.data", "w") as file:
    file.write("\n".join(train))
    
with open("valid.data", "w") as file:
    file.write("\n".join(valid)) 

len(train), len(valid)    

## FINE-TUNING

In [ ]:
# To fit all the data in Collab GPU, we use Medium model with block size of 1024
# If your GPU has more than 16Gb, go better with Large and 2048 blocks

!python pretrain_transformers.py \
    --output_dir=model \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3medium_based_on_gpt2 \
    --do_train \
    --train_data_file=train.data \
    --do_eval \
    --eval_data_file=valid.data \
    --fp16 \
    --per_gpu_train_batch_size 1 \
    --num_train_epochs 10 \
    --block_size 1024 \
    --overwrite_output_dir \
    --line_by_line 

## 42 :)

In [ ]:
!python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path=model \
    --k=50 \
    --p=0.99 \
    --length=200 \
    --temperature=0.88 \
    --num_return_sequences=3

## EXAMPLES

**Q: Что такое коронавирус? A:**

Новый коронавирус – респираторный вирус (возбудитель ОРВИ). Он передается главным образом воздушно-капельным путем в результате вдыхания капель, выделяемых из дыхательных путей больного, например при кашле или чихании, а также капель слюны или выделений из носа. Также он может распространяться, когда больной касается любой загрязненной поверхности. В этом случае заражение происходит при касании рта, носа или глаз грязными руками.

**Q: Каковы первые симптомы коронавируса? A:**

Основные симптомы коронавируса: высокая температура тела, кашель (сухой или с небольшим количеством мокроты), затрудненное дыхание, боль в мышцах, быстрая утомляемость, а также нарушение или потеря обоняния и потеря вкуса. Реже проявляются головная боль, тяжесть в грудной клетке, тошнота и рвота. В подавляющем большинстве случаев данные симптомы связаны не с коронавирусом, а с обычной ОРВИ. Поэтому для постановки диагноза следует обратиться к врачу.

**Q: Что лучше защищает от коронавируса: маска или респиратор? A:**

Хирургические маски предназначены для предотвращения попадания капель и брызг слюны или крови. Такие маски служат физическим барьером от вирусов и микробов, передающихся воздушно-капельным путем, однако они менее эффективны, поскольку в них нет защитного фильтра. Респираторы класса FFP3 необходимы при работе с особо опасными веществами, радиоактивными материалами и при угрозе контакта с вирусами и бактериями, живущими в грязных и зараженных помещениях. Именно их ВОЗ рекомендует использовать персоналу и врачам, которые контактируют с больными или потенциально больными людьми.


**Q: Какие лекарственные препараты назначают пациентам с коронавирусом? A:**

Согласно актуальным методическим рекомендациям Минздрава России, список возможных к назначению лекарственных средств для лечения заболевания у взрослых включает фавипиравир, гидроксихлорохин, мефлохин, азитромицин, лопинавир плюс ритонавир, рекомбинантный интерферон бета-1b и рекомбинантный интерферон альфа, умифеновир. Среди препаратов упреждающей противовоспалительной терапии - тоцилизумаб, сарилумаб, олокизумаб, канакинумаб, барицитониб, тофацитиниб. Среди препаратов, влияющих на ингибирование клеточного иммунного ответа - тофацитиниб, тофацитиниб плюс ритонавир, тофацитиниб с рекомбинантным интерфероном альфа, тофацитиниб с рекомбинантным интерфероном бета-1b.
